In [ ]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
eps, weight_tg, weight_x3, weight_x5 = 10**(-3), 0.0, 0.0, 0.0 # last couple =0.0
weight = 1.0 - weight_tg - weight_x3 - weight_x5

def transform(x):
    x['cp_type'] = x[x['cp_type']=='trt_cp']
    x.pop('cp_type')
    x['cp_dose'] = x['cp_dose'].replace({'D1':-1, 'D2':1})
    x['cp_time'] = x['cp_time'] // 24
    x.join(pd.get_dummies(x['cp_time']))
    x.pop('cp_time')
    return x
def cube(x):
    x /= 10.0
    return 10.0 * x*x*x

def tgfunc(x):
    return  np.tan(np.pi *x/ 4.0) 
def application_transform(x):
    x = 2*x-1.0
    x1 = weight_tg*tgfunc(x) + (weight_x3 + weight_x5*x**2)*x**3 + weight*x
    x1 = x1/2 + 0.5
    if x1 < eps:
        x1 = eps
    elif x1 >= 1 - eps:
        x1 = 1 - eps
    return x1

X_all = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
y_all = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
whatisit = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
X = transform(X_all[X_all.columns[1:]])
Y = y_all[y_all.columns[1:]]


for random_state in range(2020,2070,10):
    X_train, submission, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, 
                                                        random_state=random_state)
    pseudoinverse_X = np.linalg.pinv(X_train)
    subm = np.dot(submission,np.dot(pseudoinverse_X,Y_train))

    dim = 206*len(submission) 
    df = pd.DataFrame(subm)
    df = df.applymap(application_transform)
    df = np.array(df).reshape(dim)
    
    print(log_loss(np.array(Y_test).reshape(dim),df))

print([eps, weight, weight_tg, weight_x3, weight_x5, len(X.columns)])



In [ ]:
# argmax = []
# B = np.dot(pseudoinverse_X,Y_train)
# for i in range(B.shape[0]):
#     for j in range(B.shape[1]):
#         if B[i,j] > 0.005:
#             argmax.append([i,j])
# len(argmax)            


### Здесь происходит добавление новых признаков x_i^3
# X3 = X.applymap(cube)
# X3.columns = ['cube'+ch for ch in X3.columns]
# X = X.join(X3)

## Здесь происходит добавление новых признаков ((1.0 + x_i/10.0)*(1.0 + x_j/10.0) - 1.0)*10.0
# for t in argmax:
#     i, j = t[0], t[1]
#     X[str(i)+'-'+str(j)] = ((1.0+X[X.columns[i]]/10.0)*(1.0+X[X.columns[j]]/10.0)-2.0)*5.0 

## Здесь происходит добавление новых признаков x_i + x_j
# for t in argmax:
#     i, j = t[0], t[1]
#     X[str(i)+'-'+str(j)] = (X[X.columns[i]]*X[X.columns[j]])*0.1 

In [ ]:
np.linalg.svd(np.dot(pseudoinverse_X,Y_train))

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm

In [ ]:
def create_model(num_columns):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(num_columns),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid"))
    ])
    model.compile(optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period=10),
                  loss='binary_crossentropy', 
                  )
    return model

In [ ]:
def metric(y_true, y_pred):
    metrics = []
    for _target in train_targets.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [ ]:
N_STARTS = 7
tf.random.set_seed(42)

res = train_targets.copy()
ss.loc[:, train_targets.columns] = 0
res.loc[:, train_targets.columns] = 0

for seed in range(N_STARTS):
    for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits=7, random_state=seed, shuffle=True).split(train_targets, train_targets)):
        print(f'Fold {n}')
    
        model = create_model(len(top_feats))
        checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
        
        model.fit(train.values[tr][:, top_feats],
                  train_targets.values[tr],
                  validation_data=(train.values[te][:, top_feats], train_targets.values[te]),
                  epochs=35, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt], verbose=2
                 )
        
        model.load_weights(checkpoint_path)
        test_predict = model.predict(test.values[:, top_feats])
        val_predict = model.predict(train.values[te][:, top_feats])
        
        ss.loc[:, train_targets.columns] += test_predict
        res.loc[te, train_targets.columns] += val_predict
        print('')
    
ss.loc[:, train_targets.columns] /= ((n+1) * N_STARTS)
res.loc[:, train_targets.columns] /= N_STARTS

In [ ]:
X_all.shape

In [ ]:

for random_state in range(2020,2070,10):
    checkpoint_path = f'repeat:{random_state}.hdf5'
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
    cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                 save_weights_only = True, mode = 'min')
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, 
                                                            random_state=random_state)
    
    model = create_model(X.shape[1])
    model.fit(X_train, Y_train,
              validation_data=(X_test, Y_test), 
              epochs=35, batch_size=128,
              callbacks=[reduce_lr_loss, cb_checkpt], verbose=2)
    model.load_weights(checkpoint_path)
    test_predict = model.predict(X_test)
    
    df = pd.DataFrame(subm)
    df = df.applymap(application_transform)
    df = np.array(df).reshape(dim)
    
    print(log_loss(np.array(Y_test),test_predict))
